### 参考
1. [Spark_SQL_cheatsheet_code.ipynb](https://github.com/ShowMeAI-Hub/awesome-AI-cheatsheets/blob/main/Spark/Spark_SQL_cheatsheet_code.ipynb)， 镜像在gitee [ShowMeAI-Hub--awesome-AI-cheatsheets.git](https://gitee.com/mirrr/ShowMeAI-Hub--awesome-AI-cheatsheets/tree/main/Spark)

### 环境安装

https://cdn.azul.com/zulu/bin/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz


```shell
source /app/Miniconda3-py37_4.12.0/bin/activate
cd /fridaAnlzAp/analyze_by_spark_GraphFrame/
jupyter notebook &
```


### chrome ipynb 阅读器

chrome ipynb 阅读器， https://github.com/imvladikon/jupyter-notebook-viewer?tab=readme-ov-file

https://chrome.google.com/webstore/detail/jupyter-notebook-viewer/ocabfdicbcamoonfhalkdojedklfcjmf?hl=en


本文地址， http://giteaz:3000/frida_analyze_app_src/analyze_by_spark_GraphFrame/raw/branch/main/demo.ipynb

In [1]:
%%bash
date && pwd

which python
which pip

file /app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64 /app/spark-2.4.8-bin-hadoop2.7
md5sum /app/pack/spark-2.4.8-bin-hadoop2.7.tgz /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

ls -lh /fridaAnlzAp/frida_js/*RunBuszJs*.log


2024年 03月 26日 星期二 08:32:29 CST
/fridaAnlzAp/analyze_by_spark_GraphFrame
/app/Miniconda3-py37_4.12.0/bin/python
/app/Miniconda3-py37_4.12.0/bin/pip
/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64: directory
/app/spark-2.4.8-bin-hadoop2.7:             directory
3af9ae26e91360c0f39a5647d9279715  /app/pack/spark-2.4.8-bin-hadoop2.7.tgz
3d8073a1e7bc71a0c53bbbbad590dad2  /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz
-rwxrwxrwx 1 z z 263M  3月 24 16:57 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log


In [2]:
%%bash

pip install findspark


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
import os
os.environ["JAVA_HOME"]="/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64"
# os.environ["SPARK_HOME"="/app/spark-2.4.8-bin-hadoop2.7"



In [4]:
import findspark
findspark.init(spark_home="/app/spark-2.4.8-bin-hadoop2.7")

In [5]:
%%bash

env | grep HOME

JAVA_HOME=/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64
HOME=/home/z
SPARK_HOME=/app/spark-2.4.8-bin-hadoop2.7


In [6]:
from pyspark.sql import SparkSession
#必须用py3.7 , 否则 这一行报错 TypeError: 'bytes' object cannot be interpreted as an integer

SparkSession 用于创建数据帧，将数据帧注册为表，执行 SQL 查询，缓存表及读取 Parquet 文件。

In [7]:
spark = SparkSession \
       .builder \
       .appName("analyze_by_spark_GraphFrame") \
       .config("spark.some.config.option", "some-value") \
       .getOrCreate()

24/03/25 23:54:23 WARN Utils: Your hostname, mchr resolves to a loopback address: 127.0.1.1; using 10.0.4.23 instead (on interface wlo1)
24/03/25 23:54:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/25 23:54:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 23:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *

###  从Spark数据源创建


#### JSON

In [9]:
%%bash

head -n 2 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log

{"curThreadId":392463,"direct":1,"fnAdr":"0x55555556b4fe","fnCallId":1,"fnSym":{"address":"0x55555556b4fe","name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev","moduleName":"simple_nn.elf","fileName":"","lineNumber":0,"column":0}}
{"curThreadId":392463,"direct":1,"fnAdr":"0x55555556b752","fnCallId":2,"fnSym":{"address":"0x55555556b752","name":"_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEEEC1Ev","moduleName":"simple_nn.elf","fileName":"","lineNumber":0,"column":0}}


In [10]:
df0 = spark.read.json("/fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log")


### 查阅数据信息

#### 查阅spark Dataframe的信息

In [11]:
df0.columns #返回 df 的列名

['curThreadId', 'direct', 'fnAdr', 'fnCallId', 'fnSym']

In [12]:
df0.dtypes   #返回 df 的列名与数据类型

[('curThreadId', 'bigint'),
 ('direct', 'bigint'),
 ('fnAdr', 'string'),
 ('fnCallId', 'bigint'),
 ('fnSym',
  'struct<address:string,column:bigint,fileName:string,lineNumber:bigint,moduleName:string,name:string>')]

In [13]:
df0.printSchema() #返回 df的 Schema

root
 |-- curThreadId: long (nullable = true)
 |-- direct: long (nullable = true)
 |-- fnAdr: string (nullable = true)
 |-- fnCallId: long (nullable = true)
 |-- fnSym: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- column: long (nullable = true)
 |    |-- fileName: string (nullable = true)
 |    |-- lineNumber: long (nullable = true)
 |    |-- moduleName: string (nullable = true)
 |    |-- name: string (nullable = true)



In [14]:
df0.schema #返回 df 的 Schema

StructType(List(StructField(curThreadId,LongType,true),StructField(direct,LongType,true),StructField(fnAdr,StringType,true),StructField(fnCallId,LongType,true),StructField(fnSym,StructType(List(StructField(address,StringType,true),StructField(column,LongType,true),StructField(fileName,StringType,true),StructField(lineNumber,LongType,true),StructField(moduleName,StringType,true),StructField(name,StringType,true))),true)))

In [15]:
df0.describe().show()  #汇总统计数据

[Stage 1:===>                                                     (1 + 15) / 16]

+-------+-----------+------------------+--------------+------------------+
|summary|curThreadId|            direct|         fnAdr|          fnCallId|
+-------+-----------+------------------+--------------+------------------+
|  count|    1045003|           1045003|       1045003|           1045003|
|   mean|   392463.0|1.4999995215324742|          null|261251.25001746407|
| stddev|        0.0|0.5000002392337048|          null|150833.26303729884|
|    min|     392463|                 1|0x55555556658f|                 1|
|    max|     392463|                 2|0x7ffff7807ad4|            522502|
+-------+-----------+------------------+--------------+------------------+



In [16]:
df0.show() #显示 df 的内容

+-----------+------+--------------+--------+--------------------+
|curThreadId|direct|         fnAdr|fnCallId|               fnSym|
+-----------+------+--------------+--------+--------------------+
|     392463|     1|0x55555556b4fe|       1|[0x55555556b4fe, ...|
|     392463|     1|0x55555556b752|       2|[0x55555556b752, ...|
|     392463|     1|0x55555556cc0c|       3|[0x55555556cc0c, ...|
|     392463|     2|0x55555556cc0c|       3|[0x55555556cc0c, ...|
|     392463|     2|0x55555556b752|       2|[0x55555556b752, ...|
|     392463|     1|0x55555556ca72|       4|[0x55555556ca72, ...|
|     392463|     2|0x55555556ca72|       4|[0x55555556ca72, ...|
|     392463|     2|0x55555556b4fe|       1|[0x55555556b4fe, ...|
|     392463|     1|0x7ffff770930c|       5|[0x7ffff770930c, ...|
|     392463|     1|0x7ffff770ef00|       6|[0x7ffff770ef00, ...|
|     392463|     1|0x7ffff771130c|       7|[0x7ffff771130c, ...|
|     392463|     1|0x7ffff6d19966|       8|[0x7ffff6d19966, ...|
|     3924

In [17]:
df0.count() #返回 df 的行数

1045003

In [18]:
df0.distinct().count() #返回 df 中不重复的行数

1045003

In [19]:
df0.explain() #返回逻辑与实体方案

== Physical Plan ==
*(1) FileScan json [curThreadId#6L,direct#7L,fnAdr#8,fnCallId#9L,fnSym#10] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<curThreadId:bigint,direct:bigint,fnAdr:string,fnCallId:bigint,fnSym:struct<address:string,...


In [20]:
df1 = df0.withColumn('fnSym_address',df0.fnSym.address) \
 .withColumn('fnSym_column',df0.fnSym.column) \
 .withColumn('fnSym_fileName',regexp_replace(df0.fnSym.fileName, '/home/z/torch-repo/pytorch/', './')) \
 .withColumn('fnSym_lineNumber',df0.fnSym.lineNumber) \
 .withColumn('fnSym_moduleName', df0.fnSym.moduleName) \
 .withColumn('fnSym_name', df0.fnSym["name"])

In [21]:
df1.describe().show(vertical=True)  #汇总统计数据

[Stage 9:=====================>                                   (6 + 10) / 16]

-RECORD 0--------------------------------
 summary          | count                
 curThreadId      | 1045003              
 direct           | 1045003              
 fnAdr            | 1045003              
 fnCallId         | 1045003              
 fnSym_address    | 1045003              
 fnSym_column     | 1045003              
 fnSym_fileName   | 1045003              
 fnSym_lineNumber | 1045003              
 fnSym_moduleName | 1045003              
 fnSym_name       | 1045003              
-RECORD 1--------------------------------
 summary          | mean                 
 curThreadId      | 392463.0             
 direct           | 1.4999995215324742   
 fnAdr            | null                 
 fnCallId         | 261251.25001746407   
 fnSym_address    | null                 
 fnSym_column     | 7.957558016579856    
 fnSym_fileName   | null                 
 fnSym_lineNumber | 347.8104349939665    
 fnSym_moduleName | null                 
 fnSym_name       | null          

In [22]:
df1.show(vertical=True,n=2) #显示 df 的内容

-RECORD 0--------------------------------
 curThreadId      | 392463               
 direct           | 1                    
 fnAdr            | 0x55555556b4fe       
 fnCallId         | 1                    
 fnSym            | [0x55555556b4fe, ... 
 fnSym_address    | 0x55555556b4fe       
 fnSym_column     | 0                    
 fnSym_fileName   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSt12_Vector_ba... 
-RECORD 1--------------------------------
 curThreadId      | 392463               
 direct           | 1                    
 fnAdr            | 0x55555556b752       
 fnCallId         | 2                    
 fnSym            | [0x55555556b752, ... 
 fnSym_address    | 0x55555556b752       
 fnSym_column     | 0                    
 fnSym_fileName   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSaISt10shar

### 重复值

#### dropDuplicates函数

### 查询

#### 自定义函数F

In [23]:
from pyspark.sql import functions as F

#### Select

In [24]:
df1.select("fnSym_moduleName").distinct().show() 

+----------------+
|fnSym_moduleName|
+----------------+
|   libtorch.so.1|
|       libc10.so|
|    libcaffe2.so|
|   simple_nn.elf|
+----------------+



In [25]:
df1.select("fnSym_fileName").distinct().collect()[:10]

[Row(fnSym_fileName='./aten/src/ATen/native/cpu/avx_mathfun.h'),
 Row(fnSym_fileName='./aten/src/ATen/Context.h'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/shared_ptr.h'),
 Row(fnSym_fileName='./third_party/cpuinfo/src/init.c'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/stl_algobase.h'),
 Row(fnSym_fileName='./torch/csrc/autograd/variable.cpp'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/stl_uninitialized.h'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/char_traits.h'),
 Row(fnSym_fileName='./aten/src/ATen/native/TensorShape.cpp'),
 Row(fnSym_fileName='./c10/core/Storage.h')]

In [26]:
df1.select("fnSym_name","fnSym_address").show(n=3)

+--------------------+--------------+
|          fnSym_name| fnSym_address|
+--------------------+--------------+
|_ZNSt12_Vector_ba...|0x55555556b4fe|
|_ZNSaISt10shared_...|0x55555556b752|
|_ZN9__gnu_cxx13ne...|0x55555556cc0c|
+--------------------+--------------+
only showing top 3 rows



In [27]:
#字段direct取别名"进或出"
df1.select( "fnCallId", "fnSym_address", df1.direct .alias("进或出")  ) \
.show(n=4)   

+--------+--------------+------+
|fnCallId| fnSym_address|进或出|
+--------+--------------+------+
|       1|0x55555556b4fe|     1|
|       2|0x55555556b752|     1|
|       3|0x55555556cc0c|     1|
|       3|0x55555556cc0c|     2|
+--------+--------------+------+
only showing top 4 rows



In [28]:
df1.select(df1["fnCallId"], \
  F.when(df1.direct==1, "进入函数") .when(df1.direct==2, "退出函数") .otherwise("错误值") .alias("方向")
 ).show(n=5)   
#列direct显示为人类可读

+--------+--------+
|fnCallId|    方向|
+--------+--------+
|       1|进入函数|
|       2|进入函数|
|       3|进入函数|
|       3|退出函数|
|       2|退出函数|
+--------+--------+
only showing top 5 rows



In [29]:
df1.select(df1['direct'] == 1).show(n=10)  
#显示 函数进入 

+------------+
|(direct = 1)|
+------------+
|        true|
|        true|
|        true|
|       false|
|       false|
|        true|
|       false|
|       false|
|        true|
|        true|
+------------+
only showing top 10 rows



#### When 

In [30]:
df1[df1.fnSym_fileName.isin(\
'./torch/csrc/jit/tracer.cpp',\
'./aten/src/ATen/core/Tensor.cpp')].collect() [:3]
# 显示符合指定条件的 firstName 列 的记录

[Row(curThreadId=392463, direct=1, fnAdr='0x7ffff766951f', fnCallId=111, fnSym=Row(address='0x7ffff766951f', column=56, fileName='/home/z/torch-repo/pytorch/torch/csrc/jit/tracer.cpp', lineNumber=143, moduleName='libtorch.so.1', name='getTracingState'), fnSym_address='0x7ffff766951f', fnSym_column=56, fnSym_fileName='./torch/csrc/jit/tracer.cpp', fnSym_lineNumber=143, fnSym_moduleName='libtorch.so.1', fnSym_name='getTracingState'),
 Row(curThreadId=392463, direct=1, fnAdr='0x7ffff766a41d', fnCallId=112, fnSym=Row(address='0x7ffff766a41d', column=3, fileName='/home/z/torch-repo/pytorch/torch/csrc/jit/tracer.cpp', lineNumber=254, moduleName='libtorch.so.1', name='__tls_init'), fnSym_address='0x7ffff766a41d', fnSym_column=3, fnSym_fileName='./torch/csrc/jit/tracer.cpp', fnSym_lineNumber=254, fnSym_moduleName='libtorch.so.1', fnSym_name='__tls_init'),
 Row(curThreadId=392463, direct=2, fnAdr='0x7ffff766a41d', fnCallId=112, fnSym=Row(address='0x7ffff766a41d', column=3, fileName='/home/z/tor

#### Like 

In [31]:
df1.select("fnCallId", "fnSym_address",
df1.fnSym_fileName.like("./aten/src/ATen/core")  ) \
.show(n=4)   
# 显示 fnSym_fileName 列中包含 '/home/z/torch-repo/pytorch/aten/src/ATen/core.cpp" 的 "fnCallId"、 "fnSym_address" 列的记录

+--------+--------------+----------------------------------------+
|fnCallId| fnSym_address|fnSym_fileName LIKE ./aten/src/ATen/core|
+--------+--------------+----------------------------------------+
|       1|0x55555556b4fe|                                   false|
|       2|0x55555556b752|                                   false|
|       3|0x55555556cc0c|                                   false|
|       3|0x55555556cc0c|                                   false|
+--------+--------------+----------------------------------------+
only showing top 4 rows



####  Startswith - Endswith

In [32]:
df1.select("fnCallId", "fnSym_address",
df1.fnSym_fileName.startswith("./aten/src/ATen/core").alias("以./aten/...开头的文件名") ) \
.show(n=5)  
#显示 fnSym_fileName 列以 "./aten/src/ATen/core" 开头的 "fnCallId"、 "fnSym_address" 列的记录


+--------+--------------+------------------------+
|fnCallId| fnSym_address|以./aten/...开头的文件名|
+--------+--------------+------------------------+
|       1|0x55555556b4fe|                   false|
|       2|0x55555556b752|                   false|
|       3|0x55555556cc0c|                   false|
|       3|0x55555556cc0c|                   false|
|       2|0x55555556b752|                   false|
+--------+--------------+------------------------+
only showing top 5 rows



In [33]:
df1.select( df1.fnSym_fileName .endswith("Tensor.cpp").alias("以Tensor.cpp结尾的文件名") ) \
.show(n=4)   
# 显示以 "Tensor.cpp" 结尾的 

+------------------------+
|以Tensor.cpp结尾的文件名|
+------------------------+
|                   false|
|                   false|
|                   false|
|                   false|
+------------------------+
only showing top 4 rows



#### Substring 

In [34]:
df1.select( df1.fnSym_fileName.substr(1, 1)  .alias("文件路径第一个字符") ) \
 .collect()   [15:20]


[Row(文件路径第一个字符='/'),
 Row(文件路径第一个字符='.'),
 Row(文件路径第一个字符='.'),
 Row(文件路径第一个字符='/'),
 Row(文件路径第一个字符='/')]

#### Between 

In [35]:
df1.select( df1.fnCallId.between(1, 4).alias("给定范围的调用id") ) \
.show(n=10)   
#显示介于22岁至24岁之间的 age 列的记录

+----------------+
|给定范围的调用id|
+----------------+
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|           false|
|           false|
+----------------+
only showing top 10 rows



###  添加、修改、删除列

In [36]:
df1\
.withColumnRenamed('fnCallId', '函数调用id') \
.withColumnRenamed('fnSym_fileName', '函数所在文件名') \
.drop("fnSym_address", "curThreadId") \
.show(n=2,vertical=True)

-RECORD 0--------------------------------
 direct           | 1                    
 fnAdr            | 0x55555556b4fe       
 函数调用id       | 1                    
 fnSym            | [0x55555556b4fe, ... 
 fnSym_column     | 0                    
 函数所在文件名   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSt12_Vector_ba... 
-RECORD 1--------------------------------
 direct           | 1                    
 fnAdr            | 0x55555556b752       
 函数调用id       | 2                    
 fnSym            | [0x55555556b752, ... 
 fnSym_column     | 0                    
 函数所在文件名   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSaISt10shared_... 
only showing top 2 rows



### 分组

In [37]:
df1.groupBy("direct")\
  .count() \
  .show()  
#按 direct 列分组，统计每组记录数

+------+------+
|direct| count|
+------+------+
|     1|522502|
|     2|522501|
+------+------+



#####  找到 进、出 不平衡的函数调用id 

###### api

In [38]:
df1.groupBy("fnCallId")\
  .agg( count('fnCallId').alias('x') ) \
  .where(col("x")==1).show()  # where 可以换成filter

+--------+---+
|fnCallId|  x|
+--------+---+
|  522484|  1|
+--------+---+



###### sql

In [39]:
df1.createTempView("v_df1")
tmp_df = spark.sql("SELECT fnCallId,count(*) as cnt  FROM v_df1 group by fnCallId having cnt==1")
tmp_df.show()

+--------+---+
|fnCallId|cnt|
+--------+---+
|  522484|  1|
+--------+---+



#### 筛选

######  断言 ， 不 存在 fnAdr != fnSym_address 的行

In [40]:
df1.filter( df1["fnAdr"]!=df1["fnSym_address"] )  .collect()
# 

[]

#### 排序
##### sort与orderBy操作

In [41]:
df1.columns

['curThreadId',
 'direct',
 'fnAdr',
 'fnCallId',
 'fnSym',
 'fnSym_address',
 'fnSym_column',
 'fnSym_fileName',
 'fnSym_lineNumber',
 'fnSym_moduleName',
 'fnSym_name']

In [42]:
df1.sort( df1.fnSym_address.desc() ).collect()[:2]
#按函数地址降序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x7ffff7807ad4', fnCallId=451561, fnSym=Row(address='0x7ffff7807ad4', column=7, fileName='/usr/include/c++/9/ext/new_allocator.h', lineNumber=119, moduleName='libtorch.so.1', name='deallocate'), fnSym_address='0x7ffff7807ad4', fnSym_column=7, fnSym_fileName='/usr/include/c++/9/ext/new_allocator.h', fnSym_lineNumber=119, fnSym_moduleName='libtorch.so.1', fnSym_name='deallocate'),
 Row(curThreadId=392463, direct=2, fnAdr='0x7ffff7807ad4', fnCallId=451561, fnSym=Row(address='0x7ffff7807ad4', column=7, fileName='/usr/include/c++/9/ext/new_allocator.h', lineNumber=119, moduleName='libtorch.so.1', name='deallocate'), fnSym_address='0x7ffff7807ad4', fnSym_column=7, fnSym_fileName='/usr/include/c++/9/ext/new_allocator.h', fnSym_lineNumber=119, fnSym_moduleName='libtorch.so.1', fnSym_name='deallocate')]

In [43]:
df1.sort("fnCallId", ascending=False) .collect()[:2]
#按字段fnCallId升序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=2, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev')]

In [44]:
df1.orderBy(["fnCallId","fnSym_address"],ascending=[0,1])  .collect() [:3]
#按字段fnCallId降序、字段fnSym_address升序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=2, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=1, fnAdr='0x55555556f5d6', fnCallId=522501, fnSym=Row(addres

### 运行 SQL 查询
#### 将数据帧注册为视图

In [45]:
df1.createGlobalTempView("gtView_df1")

In [46]:
df1.createTempView("tView_df1")

In [47]:
df1.createOrReplaceTempView("tView_df1")

#### 查询视图

In [48]:
tmp_df = spark.sql("SELECT * FROM tView_df1")
tmp_df.show(n=2)

+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
|curThreadId|direct|         fnAdr|fnCallId|               fnSym| fnSym_address|fnSym_column|fnSym_fileName|fnSym_lineNumber|fnSym_moduleName|          fnSym_name|
+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
|     392463|     1|0x55555556b4fe|       1|[0x55555556b4fe, ...|0x55555556b4fe|           0|              |               0|   simple_nn.elf|_ZNSt12_Vector_ba...|
|     392463|     1|0x55555556b752|       2|[0x55555556b752, ...|0x55555556b752|           0|              |               0|   simple_nn.elf|_ZNSaISt10shared_...|
+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
only showing top

### 输出
#### 数据结构

In [49]:
rdd1 = df1.rdd   #将 df 转换为 RDD
print(type(rdd1))
rdd1.first()

<class 'pyspark.rdd.RDD'>


Row(curThreadId=392463, direct=1, fnAdr='0x55555556b4fe', fnCallId=1, fnSym=Row(address='0x55555556b4fe', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev'), fnSym_address='0x55555556b4fe', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev')

In [50]:
df1Json=df1.toJSON()#将 df 转换为 RDD 字符串
print(type(df1Json))
df1Json.first()   

<class 'pyspark.rdd.RDD'>


'{"curThreadId":392463,"direct":1,"fnAdr":"0x55555556b4fe","fnCallId":1,"fnSym":{"address":"0x55555556b4fe","column":0,"fileName":"","lineNumber":0,"moduleName":"simple_nn.elf","name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev"},"fnSym_address":"0x55555556b4fe","fnSym_column":0,"fnSym_fileName":"","fnSym_lineNumber":0,"fnSym_moduleName":"simple_nn.elf","fnSym_name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev"}'

In [51]:
%%bash

pip install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [52]:
df1_pan=df1.toPandas()   #将 df 的内容转为 Pandas 的数据帧
df1_pan.head(5)

,curThreadId,direct,fnAdr,fnCallId,fnSym,fnSym_address,fnSym_column,fnSym_fileName,fnSym_lineNumber,fnSym_moduleName,fnSym_name
0,392463,1,0x55555556b4fe,1,"(0x55555556b4fe, 0, , 0, simple_nn.elf, _ZNSt1...",0x55555556b4fe,0,,0,simple_nn.elf,_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit...
1,392463,1,0x55555556b752,2,"(0x55555556b752, 0, , 0, simple_nn.elf, _ZNSaI...",0x55555556b752,0,,0,simple_nn.elf,_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEE...
2,392463,1,0x55555556cc0c,3,"(0x55555556cc0c, 0, , 0, simple_nn.elf, _ZN9__...",0x55555556cc0c,0,,0,simple_nn.elf,_ZN9__gnu_cxx13new_allocatorISt10shared_ptrIN5...
3,392463,2,0x55555556cc0c,3,"(0x55555556cc0c, 0, , 0, simple_nn.elf, _ZN9__...",0x55555556cc0c,0,,0,simple_nn.elf,_ZN9__gnu_cxx13new_allocatorISt10shared_ptrIN5...
4,392463,2,0x55555556b752,2,"(0x55555556b752, 0, , 0, simple_nn.elf, _ZNSaI...",0x55555556b752,0,,0,simple_nn.elf,_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEE...


### 保存至文件

In [53]:
df1.columns

['curThreadId',
 'direct',
 'fnAdr',
 'fnCallId',
 'fnSym',
 'fnSym_address',
 'fnSym_column',
 'fnSym_fileName',
 'fnSym_lineNumber',
 'fnSym_moduleName',
 'fnSym_name']

In [54]:
df1.select("fnCallId", "fnSym_address") .write \
  .save("fnCallId__fnSym_address.parquet",mode='overwrite')

24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 88.37% for 8 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 78.55% for 9 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 70.69% for 10 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 64.27% for 11 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 58.91% for 12 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830,196 bytes) of heap memory
Scaling row group sizes to 54.38% for 13 writers
24/03/25 23:55:18 WARN MemoryManager: Total allocation exceeds 95.00% (948,830

In [55]:
df1.select("fnCallId", "fnSym_address") .write \
  .save("fnCallId__fnSym_address.json",format="json",mode='overwrite')

### 终止SparkSession

In [56]:
spark.stop()